The Goal is to predict weapon type used based on the year, target type, country, and attack type

In [44]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
import sklearn.model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
# Read files
df = pd.read_excel('files/GTD/globalterrorismdb_0522dist.xlsx')

In [40]:
df_interest = df[['iyear', 'country_txt', 'attacktype1', 'targtype1', 'weaptype1_txt']]
df_cleaned = df_interest[df_interest['weaptype1_txt'] != 'Unknown']
df_cleaned = df_cleaned.dropna().reset_index()
display(df_cleaned)

,index,iyear,country_txt,attacktype1,targtype1,weaptype1_txt
0,3,1970,Greece,3,7,Explosives
1,4,1970,Japan,7,7,Incendiary
2,5,1970,United States,2,3,Firearms
3,6,1970,Uruguay,1,3,Firearms
4,7,1970,United States,3,21,Explosives
...,...,...,...,...,...,...
189498,209701,2020,Yemen,3,14,Explosives
189499,209702,2020,Yemen,3,14,Explosives
189500,209703,2020,Germany,7,4,Incendiary
189501,209704,2020,Afghanistan,2,14,Firearms


In [53]:
df_encoded = pd.get_dummies(df_cleaned, columns=['country_txt', 'attacktype1', 'targtype1'])

X = df_encoded.drop('weaptype1_txt', axis=1)
y = df_encoded['weaptype1_txt']
X_intercept = sm.add_constant(X)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_intercept, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=5).fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5486662621039022


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
'''
enc = OneHotEncoder()
# Transforming categorical values to matrix
y = enc.fit_transform(y.values.reshape(-1,1))

x_cat = x['country_txt']
x_cat = enc.fit_transform(x_cat.values.reshape(-1,1))
x_cat = x_cat.toarray()

x_num = x.drop(columns={'country_txt'})
x_num = x_num.values

X = np.concatenate([x_cat, x_num], axis = 1)
'''